## Loading of modules and packages

Activate the Julia environment of the project.

In [1]:
using Pkg
Pkg.activate("..\\")

  Activating project at `C:\Users\Lenovo\Documents\GitHub\Tirocinio`


Load the necessary packages.

In [2]:
using ArchGDAL
using GeoStats
using Plots

Load the necessary modules.

In [3]:
include("..\\src\\Data Gathering\\GroundData.jl")
include("..\\src\\Data Gathering\\SatelliteData.jl")

include("..\\src\\Analysis\\Aquifers.jl")
include("..\\src\\Analysis\\Lakes.jl")
include("..\\src\\Analysis\\Noises.jl")
include("..\\src\\Analysis\\Plumes.jl")
include("..\\src\\Analysis\\Sediments.jl")

Main.Sediments


Aliasing of the modules to shorten the code.

In [4]:
# Modules
const agd = ArchGDAL

# Data gathering
const grdt = GroundData
const stdt = SatelliteData

Main.SatelliteData

## Code testing 

#### Data Gathering testing

In [ ]:
# DataFrame containing the gathered data, DataFrame containing the data that is not geolocalized. 
resmt, mresmt = grdt.getGroundData(:METEO, grdt.AA, grdt.FVG, grdt.L, grdt.T, grdt.V)

In [ ]:
# Parameters measured by the stations
unique(resmt.parameter)

In [ ]:
# All relative humidity measurements
measurements = resmt[ resmt.parameter .== "Umidità relativa", : ]

# All the measurement's values
values = ( val = measurements.value, )

# All the coordinates of the measurements
coord = Tuple{Float64, Float64}[ (c[1], c[2]) for c in eachrow(measurements[:, [:longitude, :latitude]]) ]

In [ ]:
#= Based on:
    https://juliaearth.github.io/GeoStats.jl/stable/
=#

# Georeference data
D = georef(values, coord)

# Estimation domain
G = CartesianGrid(100, 100)

# Estimation problem
problem = EstimationProblem(D, G, :val)

# Solver from the list of solvers
solver = Kriging( :val => ( variogram = GaussianVariogram(range=35.0), ) )

# Solving problem
solution = solve(problem, solver)

# Solution plot
contourf(solution, clabels=true)

#### Data Analysis testing

##### Initial Setup

In [5]:
# Digital Terrain Model, the main raster for the analysis, reppresents Veneto
dtm_file = "..\\resources\\Analysis data\\DTM_32.tiff"
dtm = agd.read(dtm_file)

GDAL Dataset (Driver: GTiff/GeoTIFF)
File(s): 
  ..\resources\Analysis data\DTM_32.tiff

Dataset (width x height): 7740 x 8440 (pixels)
Number of raster bands: 1
  [GA_ReadOnly] Band 1 (Gray): 7740 x 8440 (Float32)


Test area for aquifers and lakes

In [6]:
# Shapefile holding the poligon reppresenting the surface of a lake or the
 # limits of an quifer
area_file = "..\\resources\\Analysis data\\area\\area.shp"
dtm = agd.read(area_file)

GDAL Dataset (Driver: ESRI Shapefile/ESRI Shapefile)
File(s): 
  ..\resources\Analysis data\area\area.shp
  ..\resources\Analysis data\area\area.shx
  ..\resources\Analysis data\area\area.dbf
  ..\resources\Analysis data\area\area.prj

Number of feature layers: 1
  Layer 0: area (wkbPolygon)


Create a source point to test the analysis functions.

In [7]:
# Coordinates of the source
#   lat, lon = (11.930065824163105, 45.425861311724816) # WGS84
lat, lon = (726454.9302346368, 5025993.899219433)
# Path to the source shapefile
source_dir = "..\\resources\\Analysis data\\source_shapefile"
# Directory holding the `source` files
!isdir(source_dir) && mkdir(source_dir)
# Creation of a shapefile containing the point if not already existing
if "source_32.shp" ∉ readdir(source_dir)
    agd.create(
        source_dir*"\\source_32.shp",
        driver = agd.getdriver("ESRI Shapefile")
    ) do ds
        agd.createlayer(
            geom = agd.wkbPoint,
            # spatialref = agd.importEPSG(4326) # WGS84
            spatialref = agd.importEPSG(32632)
        ) do layer
            agd.createfeature(layer) do feature
                agd.setgeom!(feature, agd.createpoint(lat, lon))
            end
            agd.copy(layer, dataset=ds)
        end
    end
end
source_file = source_dir*"\\source_32.shp"
source = agd.read(source_file)

GDAL Dataset (Driver: ESRI Shapefile/ESRI Shapefile)
File(s): 
  ..\resources\Analysis data\source_shapefile\source_32.shp
  ..\resources\Analysis data\source_shapefile\source_32.shx
  ..\resources\Analysis data\source_shapefile\source_32.dbf
  ..\resources\Analysis data\source_shapefile\source_32.prj

Number of feature layers: 1
  Layer 0: source_32 (wkbPoint)


Analysis functions execution.

Pollutants diffusion in an aquifer

In [8]:
Aquifers.run_aquifer(
    dtm_file,
    source_file,
    area_file,
    "108-88-3",
    100.0,
    1000.0,
    45,
    20.0,
    "sand",
    tolerance = 2,
    time = 10,
    orthogonal_width = 10.0,
    mixing_zone_depth = 1580.0,
    algorithm = :domenico,
    output_path = "..\\resources\\Analysis results\\test_aquifer.tiff"
)

477 milliseconds


Polutants dispersion in lakes

In [9]:
Lakes.run_lake(
    dtm_file,
    source_file,
    area_file,
    2000.0,
    135,
    0.03,
    10.0,
    tolerance = 2,
    fickian_x = 4.0,
    fickian_y = 3.0,
    output_path = "..\\resources\\Analysis results\\test_lake.tiff"
)

1080 milliseconds


Noise pollution

In [ ]:
Noises.run_noise(
    dem_file = dtm_file,
    source_file = source_file,
    temperature_K = 293.15,
    relative_humidity = 0.2,
    intensity_dB = 110.0,
    frequency = 400.0,
    output_path = "..\\resources\\Analysis results\\test_noise.tiff"
)

Airborne pollutants dispersion from a stack

In [ ]:
Plumes.run_plume(
    dtm_file,
    source_file,
    "a",
    "c",
    10000.0,
    225,
    0.1,
    80.0,
    1.0,
    tolerance = 2,
    gas_velocity = 0.1,
    gas_temperature = 150.0,
    temperature = 18.0,
    output_path = "..\\resources\\Analysis results\\test_plumes.tiff"
)

Pollutants sedimentation

In [ ]:
Sediments.run_sediment(
    dtm_file,
    source_file,
    0.03,
    13.0,
    1.0,
    10.0,
    4.0,
    315,
    0.0359,
    100,
    1,
    tolerance = 2,
    output_path = "..\\resources\\Analysis results\\test_sediments.tiff"
)